# MusicLab Notebook for Data Analysis - RPPW Animations

## Test loading pickle files with the excerpt data frames

### Import libraries

In [ ]:
import os
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

### Import data

In [ ]:
# Open dict from pickle
fname = "renick_audience_excerpt_2021-06-14.pickle"
with open(fname, 'rb') as handle: 
    renick_audience_excerpt = pickle.load(handle)

In [ ]:
# Open dict from pickle
fname = "khoparzi_audience_excerpt_2021-06-14.pickle"
with open(fname, 'rb') as handle: 
    khoparzi_audience_excerpt = pickle.load(handle)

## Animation Tutorial
Try following the animation tutorial from here: https://pythonforundergradengineers.com/live-plotting-with-matplotlib.html

### Create a static plot

In [ ]:
## start with pt = 13
pt = 13
df_motion = khoparzi_audience_excerpt[pt].sort_index().loc['2020-07-10 14:39' : '2020-07-10 14:40', :] 

In [ ]:
fig, ax = plt.subplots()

ax.plot(df_motion['timestamp'], df_motion['x'])
ax.plot(df_motion['timestamp'], df_motion['y'])
ax.plot(df_motion['timestamp'], df_motion['z'])

ax.set_xlabel('Timestamp')
ax.set_ylabel('Motion')
ax.set_title('Motion over time')
    
# save and show the plot
fig.savefig('static_plot.png')
plt.show()

### Create an animated line plot

#### Prepare the data for animation

In [ ]:
#create empty lists for x and y data
x = [] 
y = []
z = []

x_motion = df_motion['x'][:100]
y_motion = df_motion['y'][:100]
z_motion = df_motion['z'][:100]

### 2D Animation

In [ ]:
# create figure and axes objects
fig, ax = plt.subplots()
# function that draws each frame of the animation

def animate(i):
    
    x.append(x_motion[i])
    y.append(y_motion[i])
    
    ax.clear()
    ax.plot(x,y)
    ax.set_xlim([min(x_motion),max(x_motion)])
    ax.set_ylim([min(y_motion),max(y_motion)])
    
# run the animation
ani = FuncAnimation(fig, animate, frames=20, interval=500, repeat=False)

plt.show()

### 3D animation

In [ ]:
import numpy as np
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

In [ ]:
data = df_motion.loc[:, "x":"z"].values.T
#data = data[:,:120] # 120 frames = 2 secnds bc sampling freq = 60 Hz

In [ ]:
### Interval needs to be in relation to the sampling frequency
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

In [ ]:
# convert freq domain to time domain
t = 1/freq #seconds
t_ms = t*1000 # ms 

In [ ]:
fig = plt.figure()
ax = p3.Axes3D(fig)

def animate(i, data, line): # animation function
    line.set_data(data[:2, i-25:i])
    line.set_3d_properties(data[2, i-25:i])

N = data.shape[1] # number cols
line, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1])

# Setting the axes properties
ax.set_xlim3d([min(x_motion),max(x_motion)])
ax.set_xlabel('X')

ax.set_ylim3d([min(y_motion), max(y_motion)])
ax.set_ylabel('Y')

ax.set_zlim3d([min(z_motion), max(z_motion)])
ax.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate, N, fargs=(data, line), interval=t_ms, blit=False) # fargs - additional arguments to pass to 
#ani.save('matplot003.gif', writer='imagemagick')
plt.show()

## Cut Audio at First Sound

In [ ]:
import librosa

In [ ]:
from librosa import display
import IPython.display as ipd
path = 'C:/Users/danasw/Documents/PhD/Research/MusicLab_Lockdown/Files_for_Research_Drive/Raw_Data/Audio/'

#### Get sampling rate

In [ ]:
from scipy.io.wavfile import read as read_wav
import os
os.chdir(path) # change to the file directory
rb_sampling_rate, data=read_wav( 'Renick_Bell.wav') # enter your filename
print('Renick Bell sr: '+str(rb_sampling_rate))
k_sampling_rate, data=read_wav( 'Khoparzi-Audio.wav') # enter your filename
print('Khoparzi sr: '+str(k_sampling_rate))

#### Load Audio

In [ ]:
khoparzi_audio, sr = librosa.load(os.path.join(path, 'Khoparzi-Audio.wav'), sr=48000) # include argument `duration=duration*60` if you wanr to cut the sound based on the motion data
ipd.Audio(khoparzi_audio, rate=sr)

#### Understanding librosa data structure
When khoparzi_audio was loaded, it is an array with dimension 73610342. This is the number of samples and can be computed by the sampling rate x minutes x seconds. 
The librosa lib can convert frames_to_time() with a function.

In [ ]:
ten_sec = sr*10 # sampling rate times 10s
ten_sec_seg = khoparzi_audio[:ten_sec]

In [ ]:
w = display.waveshow(ten_sec_seg, sr = sr)

In [ ]:
D = librosa.stft(ten_sec_seg) # STFT of audio
S_db = librosa.amplitude_to_db(np.abs(D), ref = np.max)

### ID First Sound
By hovering the mouse and shifting around the start sample to find the first instance of sound.  
It appears that it starts at 9.5 seconds

In [ ]:
nine_pt_five = int(sr*9.5)
first_sound = khoparzi_audio[nine_pt_five:ten_sec]
w = display.waveshow(first_sound, sr = sr)
# this indicates the sound starts 0.095 s after 9.5 s

In [ ]:
sound_start = 9.595 # I found this value by using the waveshow and adjusting until the sound came right on the beginning. 
sound_st_samp = int(sound_start*sr)
first_sound = khoparzi_audio[sound_st_samp:ten_sec]
w = display.waveshow(first_sound, sr = sr)

## Sync audio to animation
### Load libraries

In [ ]:
import simpleaudio # this was installed through pip because the conda distributions were not working
from pydub import AudioSegment, silence
from pydub.playback import play
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy import signal
import numpy as np
import threading
import time
from datetime import timedelta

### Load audio in pydub

In [ ]:
path = 'C:/Users/danasw/Documents/PhD/Research/MusicLab_Lockdown/Files_for_Research_Drive/Raw_Data/Audio/'
f = os.path.join(path, 'Khoparzi-Audio.wav')
khoparzi_audio = AudioSegment.from_file(file = f, 
                                  format = "wav") 

In [ ]:
# Change back to other directory
path = 'C:/Users/danasw/Documents/PhD/Research/MusicLab_Lockdown/Files_for_Research_Drive/Code/Data_Analysis/MusicLab5-master'
os.chdir(path) # change to the file directory

### Sync music recording to motion 
To do this, you just need to make sure they are cut at the same place. The first sample in the motion recording is when participants first heard the first sound.  
Therefore you need to cut the audio recording to the first sound and then you know that the beginning of the audio and end of the audio line up with the beginning and end of the motion signal

In [ ]:
sound_start_ms = sound_start * 1000  # sound_start is in seconds so converted to ms here

In [ ]:
khoparzi_performance = khoparzi_audio[sound_start_ms:]

### Select start and stop

In [ ]:
start = pd.to_datetime('2020-07-10 14:39')
stop = pd.to_datetime('2020-07-10 14:40')

In [ ]:
first_valid_index = df_motion.first_valid_index()
time_to_start = start-first_valid_index
time_to_stop = stop-first_valid_index

### Pydub indexes in milliseconds

In [ ]:
ms_to_start = int(time_to_start.total_seconds())*1000
ms_to_stop = int(time_to_stop.total_seconds())*1000

In [ ]:
audio_excerpt = khoparzi_performance[ms_to_start:ms_to_stop] # 1-minute

In [ ]:
sampling_rate = audio_excerpt.frame_rate
song_length = audio_excerpt.duration_seconds
left = audio_excerpt.split_to_mono()[0]
x = left.get_array_of_samples()

In [ ]:
# Save audio excerpt
out = audio_excerpt.export("test_audio.wav", format = "wav")
out.close()

### Animation of Spectrogram with music
Got the code from here to make example of music and animation sync: https://stackoverflow.com/questions/61109473/synchronizing-audio-and-animation-in-python

In [ ]:
wav_file = audio_excerpt

# Fourier transform
f, t, Zxx = signal.stft(x, fs=sampling_rate, nperseg=8820, noverlap=5292)
y = np.abs(Zxx.transpose())

# Setup a separate thread to play the music
music_thread = threading.Thread(target=play, args=(wav_file,))

# Build the figure
fig = plt.figure(figsize=(14, 6))
plt.style.use('seaborn-bright')
ax = plt.axes(xlim=[0, 4000], ylim=[0, 3000])
line1, = ax.plot([], [])

# Matplotlib function to initialize animation
def init():
    global annotation1, annotation2
    line1.set_data([], [])
    annotation1 = plt.annotate("Music: {}".format(""), xy=(0.2, 0.8), xycoords='figure fraction')
    annotation2 = plt.annotate("Animation: {}".format(""), xy=(0.6, 0.8), xycoords='figure fraction')
    return line1,

# Function for the animation
def animate(i):
    
    global music_start, annotation1, annotation2
    
    if i == 0:
        music_thread.start()
        music_start = time.perf_counter()
        
    i = round((time.perf_counter() - music_start)/song_length * t.size)
    line1.set_data(f, y[i])
    
    annotation1.set_text("Music: {}".format(timedelta(seconds=(time.perf_counter() - music_start))))
    annotation2.set_text("Animation: {}".format(timedelta(seconds=i / t.size * song_length)))
    return line1,

anim = FuncAnimation(fig, animate, init_func=init, interval=55)
plt.show()

### Make 3D animation of movement with audio signal 

Synchronize the audio and motion

In [ ]:
df_motion = khoparzi_audience_excerpt[pt]
motion_excerpt = df_motion[start:stop]

In [ ]:
### Interval needs to be in relation to the sampling frequency
# estimate avg sampling rate
freq = int(round(1 / np.mean(np.diff(motion_excerpt.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

In [ ]:
# convert freq domain to time domain
t = 1/freq #seconds
t_ms = t*1000 # ms 

In [ ]:
# Motion
data = motion_excerpt.loc[:, "x":"z"].values.T

x_motion = data[0]
y_motion = data[1]
z_motion = data[2]

In [ ]:
# Setup a separate thread to play the music
audio = audio_excerpt

### Run this code for audio and video in python. 

In [ ]:
music_thread = threading.Thread(target=play, args=(audio,))
fig = plt.figure()
ax = p3.Axes3D(fig)

def animate(i, data, line): # animation function
    global music_start
    
    if i == 0:
        music_thread.start()
        music_start = time.perf_counter()
        
    line.set_data(data[:2, i-25:i])
    line.set_3d_properties(data[2, i-25:i])

N = data.shape[1] # number cols
line, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1])

# Setting the axes properties
ax.set_xlim3d([min(x_motion),max(x_motion)])
ax.set_xlabel('X')

ax.set_ylim3d([min(y_motion), max(y_motion)])
ax.set_ylabel('Y')

ax.set_zlim3d([min(z_motion), max(z_motion)])
ax.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate, N, fargs=(data, line), interval=t_ms, blit=False) # fargs - additional arguments to pass to animate

plt.show()

### Run this code for saving the video without the music. 

In [ ]:
### Try FuncAnim without music
fig = plt.figure()
ax = p3.Axes3D(fig)

def animate(i, data, line): # animation function
           
    line.set_data(data[:2, i-25:i])
    line.set_3d_properties(data[2, i-25:i])

N = data.shape[1] # number cols
line, = ax.plot(data[0, 0:1], data[1, 0:1], data[2, 0:1])

# Setting the axes properties
ax.set_xlim3d([min(x_motion),max(x_motion)])
ax.set_xlabel('X')

ax.set_ylim3d([min(y_motion), max(y_motion)])
ax.set_ylabel('Y')

ax.set_zlim3d([min(z_motion), max(z_motion)])
ax.set_zlabel('Z')

ani = animation.FuncAnimation(fig, animate, N, fargs=(data, line), interval=t_ms, blit=False) # fargs - additional arguments to pass to animate

#plt.show()

In [ ]:
#Writer = animation.writers['ffmpeg']
writer = animation.FFMpegWriter(dpi = 72)

In [ ]:
ani.save('test.mp4', writer=writer) # this returned a 1-min video

## Add the audio to the video with MoviePy
It seems that including audio in matplotlib animation may not be possible with saving. Therefore you could try using moviepy

In [ ]:
# first save audio clip\

In [ ]:
from moviepy.editor import *

In [ ]:
### Import saved video 
videoclip = VideoFileClip("test.mp4")

In [ ]:
### Import audio
audioclip = AudioFileClip("test_audio.wav")

In [ ]:
videoclip2 = videoclip.set_audio(audioclip)

In [ ]:
videoclip2.write_videofile("test2.mp4", fps = 24, codec = 'mpeg4')

In [ ]:
videoclip.duration
